In [1]:
import math
import pandas as pd
import numpy as np
import scipy
import random
import math 
from scipy import stats
from csv import reader
random.seed(120)

In [2]:
path1 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/historical parameters.csv"
path2 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_instrument.csv"
path3 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_exchange.csv"
path4 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_interest.csv"
path5 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/tenor.csv"
path6 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/transaction_bond_hist_d.csv"
path7 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/spread_rate.csv"


In [3]:
def getData(path, fileext = None):
    if fileext == ".csv":
        df = pd.read_csv(path)
    elif fileext == ".xlsx" or fileext == ".xls":
        df = pd.read_excel(path)
    else:
        df = pd.read_csv(path)
    return df

In [6]:
hp = getData(path1)
n = hp['no_of_days']
n = list(n)
n = n[0]
decay_factor = hp['decay_factor']
holding_period = hp['holding_period']
confidence_level = hp['confidence_level']
decay_factor = list(decay_factor)
decay_factor = decay_factor[0]

In [22]:
hp

,decay_factor,holding_period,confidence_level,no_of_days,is_scenario_stressed
0,0,10,0.99,250,0


# Rates Instrument

In [7]:
##Importing dataset rates_instrument
rates_instrument= getData(path2)

In [8]:
#Function to get a specific type of data from the particular column in data(Filtering)
def loadData(df, colname, types = None):
    
    res_df = df[df[colname] == types].reset_index(drop=True)
    return res_df
    

In [9]:
rates_instrument_D = loadData(rates_instrument,'instrument_type',types = 'D')
rates_instrument_E = loadData(rates_instrument,'instrument_type',types = 'E')

# Rates Exchange

In [10]:
rates_exchange = getData(path3) 

In [11]:
rates_exchange

,date,currency_from,currency_to,rate,rate_stressed
0,2019-12-31,INR,INR,1,1
1,2019-12-30,INR,INR,1,1
2,2019-12-27,INR,INR,1,1
3,2019-12-26,INR,INR,1,1
4,2019-12-24,INR,INR,1,1
...,...,...,...,...,...
246,2018-12-26,INR,INR,1,1
247,2018-12-24,INR,INR,1,1
248,2018-12-21,INR,INR,1,1
249,2018-12-20,INR,INR,1,1


In [12]:
#Generating key
rates_exchange["key"] = rates_exchange["currency_from"] + '_' + rates_exchange["currency_to"]
rates_exchange

,date,currency_from,currency_to,rate,rate_stressed,key
0,2019-12-31,INR,INR,1,1,INR_INR
1,2019-12-30,INR,INR,1,1,INR_INR
2,2019-12-27,INR,INR,1,1,INR_INR
3,2019-12-26,INR,INR,1,1,INR_INR
4,2019-12-24,INR,INR,1,1,INR_INR
...,...,...,...,...,...,...
246,2018-12-26,INR,INR,1,1,INR_INR
247,2018-12-24,INR,INR,1,1,INR_INR
248,2018-12-21,INR,INR,1,1,INR_INR
249,2018-12-20,INR,INR,1,1,INR_INR


# Rate Interest

In [13]:
rates_interest = getData(path4)

In [14]:
#Generating key between curve and tenor
rates_interest["key"] = rates_interest["curve"] + '_' + rates_interest["tenor"]
rates_interest.head()

,date,curve,tenor,rate,rate_stressed,key
0,2019-12-31,INRZeroCurve,1m,4.95,4.95,INRZeroCurve_1m
1,2019-12-30,INRZeroCurve,1m,4.90,4.90,INRZeroCurve_1m
2,2019-12-27,INRZeroCurve,1m,4.90,4.90,INRZeroCurve_1m
3,2019-12-26,INRZeroCurve,1m,4.90,4.90,INRZeroCurve_1m
4,2019-12-24,INRZeroCurve,1m,4.90,4.90,INRZeroCurve_1m


In [15]:
##Adding current rate to the top of the rates
def sim_returns(df):
    simple_returns = pd.DataFrame([])

    for a, g in df.groupby('key'):
        
        g = g.reset_index(drop = True)
        g['simple_returns'] = g['rate']-g['rate'].shift(-1)
        #new_row = g.iloc[0].to_dict()
        #new_row['simple_returns'] = [0]
        #top_row = pd.DataFrame(new_row)
        g['simulated rates'] =( g['simple_returns'] + g['rate'].iloc[0]).shift(fill_value = g['rate'][0])
        #g = g.iloc[:-1 , :]
        #g = pd.concat([top_row, g]).reset_index(drop = True)

        simple_returns = simple_returns.append(g)
        
    
    return simple_returns

In [16]:
simulated_interest = sim_returns(rates_interest)
simulated_interest.head()

/tmp/ipykernel_326/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_326/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_326/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_326/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_326/2798524478.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

,date,curve,tenor,rate,rate_stressed,key,simple_returns,simulated rates
0,2019-12-31,INRZeroCurve,025m,5.04,5.04,INRZeroCurve_025m,0.04,5.04
1,2019-12-30,INRZeroCurve,025m,5.00,5.00,INRZeroCurve_025m,0.04,5.08
2,2019-12-27,INRZeroCurve,025m,4.96,4.96,INRZeroCurve_025m,-0.05,5.08
3,2019-12-26,INRZeroCurve,025m,5.01,5.01,INRZeroCurve_025m,-0.01,4.99
4,2019-12-24,INRZeroCurve,025m,5.02,5.02,INRZeroCurve_025m,0.03,5.03


In [17]:
simulated_interest

,date,curve,tenor,rate,rate_stressed,key,simple_returns,simulated rates
0,2019-12-31,INRZeroCurve,025m,5.040,5.040,INRZeroCurve_025m,0.040,5.040
1,2019-12-30,INRZeroCurve,025m,5.000,5.000,INRZeroCurve_025m,0.040,5.080
2,2019-12-27,INRZeroCurve,025m,4.960,4.960,INRZeroCurve_025m,-0.050,5.080
3,2019-12-26,INRZeroCurve,025m,5.010,5.010,INRZeroCurve_025m,-0.010,4.990
4,2019-12-24,INRZeroCurve,025m,5.020,5.020,INRZeroCurve_025m,0.030,5.030
...,...,...,...,...,...,...,...,...
246,2018-12-26,INRZeroCurve,9yr,7.371,7.371,INRZeroCurve_9yr,-0.002,6.760
247,2018-12-24,INRZeroCurve,9yr,7.373,7.373,INRZeroCurve_9yr,0.035,6.782
248,2018-12-21,INRZeroCurve,9yr,7.338,7.338,INRZeroCurve_9yr,0.023,6.819
249,2018-12-20,INRZeroCurve,9yr,7.315,7.315,INRZeroCurve_9yr,0.000,6.807


In [18]:
simulated_interest.iloc[250]

date                      2018-12-19
curve                   INRZeroCurve
tenor                           025m
rate                             6.6
rate_stressed                    6.6
key                INRZeroCurve_025m
simple_returns                   NaN
simulated rates                 5.03
Name: 250, dtype: object

# Tenors

In [19]:
tenors = getData(path5)


# Transaction file

In [20]:
tx_bond = getData(path6)
tx_bond.head(10)

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_tenor,lower_tenor_val,upper_tenor_val,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,050m,025m,050m,91,182,INRZeroCurve,1,1,0,1
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,100yr,075m,100yr,274,366,INRZeroCurve,1,1,0,1
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,150yr,125yr,150yr,456,547,INRZeroCurve,1,1,0,1
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,200yr,175yr,200yr,639,731,INRZeroCurve,1,1,0,1
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,250yr,225yr,250yr,821,912,INRZeroCurve,1,1,0,1
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,250yr,225yr,250yr,821,912,INRZeroCurve,1,1,0,1
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,050m,025m,050m,91,182,INRZeroCurve,1,1,0,2
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,125yr,100yr,125yr,366,456,INRZeroCurve,1,1,0,2
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,225yr,200yr,225yr,731,821,INRZeroCurve,1,1,0,2
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,325yr,300yr,325yr,1096,1186,INRZeroCurve,1,1,0,2


In [21]:
#print(tx_bond["instrument"],_ ,tx_bond["k"])
tx_bond["key"] = tx_bond['instrument'].map(str) + '_' + tx_bond['k'].map(str) 
tx_bond.head(10)

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,lower_tenor_val,upper_tenor_val,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,025m,050m,91,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,075m,100yr,274,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,125yr,150yr,456,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,175yr,200yr,639,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,225yr,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,225yr,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,025m,050m,91,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,100yr,125yr,366,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,200yr,225yr,731,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,300yr,325yr,1096,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4


In [23]:
##Sorting data by'key' column
tx_bond= tx_bond.sort_values(by=['key']).reset_index(drop=True)
tx_bond.head(10)

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,lower_tenor_val,upper_tenor_val,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,025m,050m,91,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,075m,100yr,274,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,125yr,150yr,456,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,175yr,200yr,639,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,225yr,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,225yr,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,025m,050m,91,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,100yr,125yr,366,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,200yr,225yr,731,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,300yr,325yr,1096,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4


In [24]:
#Converting to correct datetime format
tx_bond["c_date"] = pd.to_datetime(tx_bond["current_date"],format =  "%d-%m-%Y")
tx_bond.head(10)

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_tenor_val,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,050m,91,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1,2020-04-20
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,100yr,274,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2,2020-04-20
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,150yr,456,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3,2020-04-20
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,200yr,639,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4,2020-04-20
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5,2020-04-20
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,250yr,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6,2020-04-20
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,050m,91,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1,2020-04-20
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,125yr,366,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2,2020-04-20
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,225yr,731,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3,2020-04-20
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,325yr,1096,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4,2020-04-20


In [25]:
#Adding c_date and residual_maturity_expected column
tx_bond['m_date_E'] = tx_bond['c_date'] + pd.to_timedelta(tx_bond['residual_maturity_expected'], unit='d')
tx_bond.head(10)

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,lower_maturity,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,91,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,274,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,456,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,639,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,821,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,91,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,366,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,731,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3,2020-04-20,2022-08-15
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,1096,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4,2020-04-20,2023-08-15


In [26]:
#Adding c_date and residual_maturity_Actual column
tx_bond['m_date_A'] = tx_bond['c_date'] + pd.to_timedelta(tx_bond['residual_maturity_Actual'], unit='d')
tx_bond

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_maturity,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,182,INRZeroCurve,1,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,366,INRZeroCurve,1,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,547,INRZeroCurve,1,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,731,INRZeroCurve,1,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,912,INRZeroCurve,1,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,182,INRZeroCurve,1,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,456,INRZeroCurve,1,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,821,INRZeroCurve,1,1,0,2,IN1520190101NCD_3,2020-04-20,2022-08-15,2022-08-15
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,1186,INRZeroCurve,1,1,0,2,IN1520190101NCD_4,2020-04-20,2023-08-15,2023-08-15


In [27]:
tx_bond["lower_tenor_key"] = tx_bond["curve"]+ "_" + tx_bond["lower_tenor"]
tx_bond.head()

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,split_curve,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,INRZeroCurve,1,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr


In [28]:
tx_bond["upper_tenor_key"] = tx_bond["curve"]+ "_" + tx_bond["upper_tenor"]
tx_bond

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,accounting_book,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,1,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,1,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,1,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,1,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,1,1,0,2,IN1520190101NCD_3,2020-04-20,2022-08-15,2022-08-15,INRZeroCurve_200yr,INRZeroCurve_225yr
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,1,1,0,2,IN1520190101NCD_4,2020-04-20,2023-08-15,2023-08-15,INRZeroCurve_300yr,INRZeroCurve_325yr


In [29]:
lower_tenor_key_list = list(tx_bond["lower_tenor_key"])
upper_tenor_key_list = list(tx_bond["upper_tenor_key"])
tenor_list = lower_tenor_key_list + upper_tenor_key_list
tenor_list = set(tenor_list)
tenor_list = list(tenor_list)
tenor_df = pd.DataFrame(tenor_list).reset_index(drop = True)
tenor_df = tenor_df.rename(columns = {0:'tenor_list'})

In [30]:
tenor_df

,tenor_list
0,INRZeroCurve_050m
1,INRZeroCurve_200yr
2,INRZeroCurve_150yr
3,INRZeroCurve_075m
4,INRZeroCurve_225yr
5,INRZeroCurve_025m
6,INRZeroCurve_175yr
7,INRZeroCurve_125yr
8,INRZeroCurve_300yr
9,INRZeroCurve_250yr


In [31]:
#tenor_list_unique = tenor_list_unique.rename(columns = {0:'tenor_list_unique'})
tenor_df = tenor_df.sort_values('tenor_list')
tenor_df = tenor_df.reset_index(drop=True)

Matching the key values of simulated_interest dataframe with the tenor_list_unique key values of tenor_list_unique dataframe 
and dropped out the columns 'key','date','rate_stressed','simple_returns','simulated rates'.

In [32]:
rates_interest_current = pd.merge(tenor_df,simulated_interest,how='left',left_on='tenor_list',right_on='key').drop(['key','date','rate_stressed','simple_returns','rate'], axis=1)

In [33]:
rates_interest_current.head(50)

,tenor_list,curve,tenor,simulated rates
0,INRZeroCurve_025m,INRZeroCurve,025m,5.04
1,INRZeroCurve_025m,INRZeroCurve,025m,5.08
2,INRZeroCurve_025m,INRZeroCurve,025m,5.08
3,INRZeroCurve_025m,INRZeroCurve,025m,4.99
4,INRZeroCurve_025m,INRZeroCurve,025m,5.03
5,INRZeroCurve_025m,INRZeroCurve,025m,5.07
6,INRZeroCurve_025m,INRZeroCurve,025m,5.02
7,INRZeroCurve_025m,INRZeroCurve,025m,5.06
8,INRZeroCurve_025m,INRZeroCurve,025m,5.03
9,INRZeroCurve_025m,INRZeroCurve,025m,5.01


In [34]:
rates_interest_current.shape

(3263, 4)

In [35]:
tenor_rates = (rates_interest_current.drop_duplicates(['tenor_list']))
tenor_rates = tenor_rates.reset_index(drop=True)
tenor_rates

,tenor_list,curve,tenor,simulated rates
0,INRZeroCurve_025m,INRZeroCurve,025m,5.040
1,INRZeroCurve_050m,INRZeroCurve,050m,5.170
2,INRZeroCurve_075m,INRZeroCurve,075m,5.190
3,INRZeroCurve_100yr,INRZeroCurve,100yr,5.190
4,INRZeroCurve_125yr,INRZeroCurve,125yr,5.190
5,INRZeroCurve_150yr,INRZeroCurve,150yr,6.110
6,INRZeroCurve_175yr,INRZeroCurve,175yr,5.190
7,INRZeroCurve_1m,INRZeroCurve,1m,4.950
8,INRZeroCurve_200yr,INRZeroCurve,200yr,5.776
9,INRZeroCurve_225yr,INRZeroCurve,225yr,5.776


In [36]:
tx_bond_new = tx_bond.merge(tenor_rates,how='inner',left_on='lower_tenor_key',right_on='tenor_list')

In [37]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,tenor_list,curve_y,tenor,simulated rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,INRZeroCurve_025m,INRZeroCurve,025m,5.040
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,INRZeroCurve_025m,INRZeroCurve,025m,5.040
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,INRZeroCurve_025m,INRZeroCurve,025m,5.040
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,INRZeroCurve_075m,INRZeroCurve,075m,5.190
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,INRZeroCurve_075m,INRZeroCurve,075m,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,INRZeroCurve_125yr,INRZeroCurve,125yr,5.190
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,INRZeroCurve_175yr,INRZeroCurve,175yr,5.190
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,INRZeroCurve_225yr,INRZeroCurve,225yr,5.776
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,INRZeroCurve_225yr,INRZeroCurve,225yr,5.776
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,INRZeroCurve_100yr,INRZeroCurve,100yr,5.190


In [38]:
tx_bond_new.shape

(17, 39)

In [39]:
list(tx_bond_new.columns)

['id',
 'instrument',
 'instrument_id',
 'k',
 'currency',
 'currency_base',
 'Cashflow',
 'Cashflow_Type',
 'Maturity_Amount',
 'Face_value',
 'curve_x',
 'current_date',
 'Cash_flow_Dates',
 'holiday',
 'Actual_Cash_flow_date',
 'maturity_date',
 'residual_maturity_expected',
 'residual_maturity_Actual',
 'lower_tenor',
 'upper_tenor',
 'lower_tenor_val',
 'upper_tenor_val',
 'lower_maturity',
 'upper_maturity',
 'split_curve',
 'accounting_book',
 'accounting_portfolio',
 'mark_up_value',
 'intbasis',
 'key',
 'c_date',
 'm_date_E',
 'm_date_A',
 'lower_tenor_key',
 'upper_tenor_key',
 'tenor_list',
 'curve_y',
 'tenor',
 'simulated rates']

In [40]:
from copy import deepcopy

In [41]:
test_rate_df = deepcopy(rates_interest_current)

test_rate_df

#(test_rate_df.drop_duplicates(['tenor_list_unique']))

,tenor_list,curve,tenor,simulated rates
0,INRZeroCurve_025m,INRZeroCurve,025m,5.040
1,INRZeroCurve_025m,INRZeroCurve,025m,5.080
2,INRZeroCurve_025m,INRZeroCurve,025m,5.080
3,INRZeroCurve_025m,INRZeroCurve,025m,4.990
4,INRZeroCurve_025m,INRZeroCurve,025m,5.030
...,...,...,...,...
3258,INRZeroCurve_325yr,INRZeroCurve,325yr,6.367
3259,INRZeroCurve_325yr,INRZeroCurve,325yr,6.307
3260,INRZeroCurve_325yr,INRZeroCurve,325yr,6.367
3261,INRZeroCurve_325yr,INRZeroCurve,325yr,6.357


In [42]:
rates_interest_current_1V = (test_rate_df.drop_duplicates(['tenor_list']))
rates_interest_current_1V = rates_interest_current_1V.reset_index(drop=True)
rates_interest_current_1V

,tenor_list,curve,tenor,simulated rates
0,INRZeroCurve_025m,INRZeroCurve,025m,5.040
1,INRZeroCurve_050m,INRZeroCurve,050m,5.170
2,INRZeroCurve_075m,INRZeroCurve,075m,5.190
3,INRZeroCurve_100yr,INRZeroCurve,100yr,5.190
4,INRZeroCurve_125yr,INRZeroCurve,125yr,5.190
5,INRZeroCurve_150yr,INRZeroCurve,150yr,6.110
6,INRZeroCurve_175yr,INRZeroCurve,175yr,5.190
7,INRZeroCurve_1m,INRZeroCurve,1m,4.950
8,INRZeroCurve_200yr,INRZeroCurve,200yr,5.776
9,INRZeroCurve_225yr,INRZeroCurve,225yr,5.776


Correct till here

In [43]:
rates_interest_current.iloc[251]

tenor_list         INRZeroCurve_050m
curve                   INRZeroCurve
tenor                           050m
simulated rates                 5.17
Name: 251, dtype: object

In [44]:
##Sorting rates_interest_current df tenor wise and reset index
# rates_interest_current= rates_interest_current.sort_values(by=['tenor'])
# rates_interest_current.reset_index(drop=True)

In [45]:
###Filtering according to Instrument
rates_interest_current_1S = rates_interest_current.filter(['tenor_list', 'simulated rates']) ##To extract particular columns
rates_interest_current_1S

,tenor_list,simulated rates
0,INRZeroCurve_025m,5.040
1,INRZeroCurve_025m,5.080
2,INRZeroCurve_025m,5.080
3,INRZeroCurve_025m,4.990
4,INRZeroCurve_025m,5.030
...,...,...
3258,INRZeroCurve_325yr,6.367
3259,INRZeroCurve_325yr,6.307
3260,INRZeroCurve_325yr,6.367
3261,INRZeroCurve_325yr,6.357


In [46]:
rates_interest_current_1S = rates_interest_current_1S.set_index('tenor_list')
rates_interest_current_1S

,simulated rates
tenor_list,
INRZeroCurve_025m,5.040
INRZeroCurve_025m,5.080
INRZeroCurve_025m,5.080
INRZeroCurve_025m,4.990
INRZeroCurve_025m,5.030
...,...
INRZeroCurve_325yr,6.367
INRZeroCurve_325yr,6.307
INRZeroCurve_325yr,6.367


In [47]:

##Fetching current values for each of the tenor_list_unique

In [48]:
##Segregating simulations according to the tenor_list_unique(unique key) generated
df = (rates_interest_current_1S.assign(labels = rates_interest_current_1S.groupby(level = 0).cumcount())
            .groupby([rates_interest_current_1S.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))
df

,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,...,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates,simulated rates
tenor_list,,,,,,,,,,,,,,,,,,,,,
INRZeroCurve_025m,5.040,5.080,5.080,4.990,5.030,5.070,5.020,5.060,5.030,5.010,...,5.020,5.050,4.900,5.060,5.110,5.040,4.960,5.040,5.190,5.030
INRZeroCurve_050m,5.170,5.140,5.200,5.180,5.130,5.180,5.150,5.190,5.180,5.150,...,5.150,5.120,5.160,5.180,5.160,5.190,5.190,5.150,5.150,5.160
INRZeroCurve_075m,5.190,5.160,5.150,5.200,5.150,5.220,5.130,5.200,5.200,5.230,...,5.190,5.190,5.190,5.190,5.190,5.190,5.190,5.190,5.190,5.190
INRZeroCurve_100yr,5.190,5.160,5.150,5.200,5.150,5.220,5.130,5.200,5.200,5.230,...,5.170,5.160,5.200,5.180,5.200,5.200,5.140,5.200,5.190,5.190
INRZeroCurve_125yr,5.190,5.160,5.150,5.200,5.150,5.220,5.130,5.200,5.200,5.230,...,5.170,5.160,5.200,5.180,5.200,5.200,5.140,5.200,5.190,5.190
INRZeroCurve_150yr,6.110,6.110,6.110,6.110,6.110,6.110,6.110,6.110,6.110,6.110,...,6.090,6.080,6.120,6.100,6.120,6.120,6.060,6.120,6.110,6.110
INRZeroCurve_175yr,5.190,5.160,5.150,5.200,5.150,5.220,5.130,5.200,5.200,5.230,...,5.170,5.160,5.200,5.180,5.200,5.200,5.140,5.200,5.190,5.190
INRZeroCurve_1m,4.950,5.000,4.950,4.950,4.950,4.950,4.880,4.970,4.970,4.910,...,4.930,4.960,4.840,4.910,4.950,4.960,5.000,4.890,5.020,4.940
INRZeroCurve_200yr,5.776,5.721,5.787,5.808,5.755,5.748,5.735,5.835,5.809,5.741,...,5.729,5.798,5.743,5.788,5.779,5.743,5.824,5.723,5.793,5.776


In [49]:
#Extracting values at 0th index
tenor_rates = df.iloc[:,0]
tenor_rates

tenor_list
INRZeroCurve_025m     5.040
INRZeroCurve_050m     5.170
INRZeroCurve_075m     5.190
INRZeroCurve_100yr    5.190
INRZeroCurve_125yr    5.190
INRZeroCurve_150yr    6.110
INRZeroCurve_175yr    5.190
INRZeroCurve_1m       4.950
INRZeroCurve_200yr    5.776
INRZeroCurve_225yr    5.776
INRZeroCurve_250yr    5.776
INRZeroCurve_300yr    6.357
INRZeroCurve_325yr    6.357
Name: simulated rates, dtype: float64

In [50]:
tenor_rates_df_V = pd.DataFrame(tenor_rates)

In [51]:
tenor_rates_df_V

,simulated rates
tenor_list,
INRZeroCurve_025m,5.040
INRZeroCurve_050m,5.170
INRZeroCurve_075m,5.190
INRZeroCurve_100yr,5.190
INRZeroCurve_125yr,5.190
INRZeroCurve_150yr,6.110
INRZeroCurve_175yr,5.190
INRZeroCurve_1m,4.950
INRZeroCurve_200yr,5.776


In [52]:
# tenor_rates_df = pd.DataFrame(tenor_rates,columns=['rate'])
# tenor_rates_df.head()

In [53]:
type(tenor_rates_df_V)

pandas.core.frame.DataFrame

In [54]:
tenor_rates_df_V.index.rename('key',inplace=True)

In [55]:
tenor_rates_df_V.head()

,simulated rates
key,
INRZeroCurve_025m,5.04
INRZeroCurve_050m,5.17
INRZeroCurve_075m,5.19
INRZeroCurve_100yr,5.19
INRZeroCurve_125yr,5.19


Match lower tenor key with the tenor_rate df and get the corresponding rate values.
Match upper tenor key with the tenor_rate df and get the corresponding rate values


In [56]:
tenor_rates_df_V

,simulated rates
key,
INRZeroCurve_025m,5.040
INRZeroCurve_050m,5.170
INRZeroCurve_075m,5.190
INRZeroCurve_100yr,5.190
INRZeroCurve_125yr,5.190
INRZeroCurve_150yr,6.110
INRZeroCurve_175yr,5.190
INRZeroCurve_1m,4.950
INRZeroCurve_200yr,5.776


In [57]:
tx_bond_new = tx_bond.merge(tenor_rates_df_V,how='inner',left_on='lower_tenor_key',right_on='key')

In [58]:
#tx_bond_new = tx_bond_new.sort_values(by = ['lower_tenor_key'])

In [59]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,simulated rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,1,0,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,1,0,1,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190


In [60]:
##Renaming the rate column
tx_bond_new = tx_bond_new.rename(columns = {'simulated rates':'lower_rates'})
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,accounting_portfolio,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,1,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,1,0,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,1,0,1,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,1,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,1,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,1,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190


In [61]:
#Upper rates
tx_bond_new = tx_bond_new.merge(tenor_rates_df_V, how='inner', left_on='upper_tenor_key', right_on='key')

In [62]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,simulated rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190


In [63]:
#tx_bond_new = tx_bond_new.sort_values(by = ['upper_tenor_key'])

In [64]:
tx_bond_new =  tx_bond_new.rename(columns = {'simulated rates':'upper_rates'})

In [65]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,mark_up_value,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,0,1,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,0,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,0,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,0,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190


In [66]:
# Interpolated rates

In [67]:
def interpolate_values(df):

    numerator1 = (df['upper_rates'] - df['lower_rates'])
    numerator2 = (df['residual_maturity_expected'] - df['lower_maturity'])
    numerator = numerator1*numerator2
    denominator = (df['upper_maturity'] - df['lower_maturity'])
    interpolate_col = df['lower_rates'] + (numerator/denominator)
    return interpolate_col


In [68]:
tx_bond_new['Interpolated_rates'] = tx_bond_new.apply(lambda x: x['upper_rates'] if x['upper_tenor_key'] == x['lower_tenor_key'] else interpolate_values(x), axis=1)

In [69]:
tx_bond_new

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.127143
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000
4,PF001,IN1520190101PF,16803,4,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_4,2020-04-20,2021-03-19,2021-03-21,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543
7,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000
8,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,1,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000


In [70]:
##Sorting the dataframe lower_rate wise
tx_bond_new.sort_values(by=['lower_rates'])

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,intbasis,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates
15,PF001,IN1520190101PF,16803,1,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_1,2020-04-20,2020-06-19,2020-06-21,INRZeroCurve_1m,INRZeroCurve_025m,4.950,5.040,4.993500
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000
1,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,2,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143
2,PF001,IN1520190101PF,16803,2,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_2,2020-04-20,2020-09-19,2020-09-20,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.127143
16,PF001,IN1520190101PF,16803,3,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_3,2020-04-20,2020-12-19,2020-12-20,INRZeroCurve_050m,INRZeroCurve_075m,5.170,5.190,5.183261
5,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538
6,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543
3,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,1,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000
9,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,2,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000
10,PF001,IN1520190101PF,16803,5,INR,INR,5000,C,200000,100,...,1,IN1520190101PF_5,2020-04-20,2021-06-19,2021-06-20,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000


### SPREADs

In [71]:
spreads= getData(path7)
spreads

,date,instrument,rate,rate_stressed
0,2019-12-31,IN1520190101NCD,5.443980,5.443980
1,2019-12-30,IN1520190101NCD,5.273412,5.273412
2,2019-12-27,IN1520190101NCD,5.299568,5.299568
3,2019-12-26,IN1520190101NCD,5.290016,5.290016
4,2019-12-24,IN1520190101NCD,5.510400,5.510400
...,...,...,...,...
1501,2018-12-26,INE134E08925PSUT,0.949293,0.949293
1502,2018-12-24,INE134E08925PSUT,0.949293,0.949293
1503,2018-12-21,INE134E08925PSUT,0.949293,0.949293
1504,2018-12-20,INE134E08925PSUT,0.949293,0.949293


In [72]:
##Dropping duolicates
spread_rates = (spreads.drop_duplicates(['instrument']))
spread_rates.columns= spread_rates.columns.str.lower() ##Column names to lower case
spread_rates.head()

,date,instrument,rate,rate_stressed
0,2019-12-31,IN1520190101NCD,5.443980,5.443980
251,2019-12-31,IN1520190101PF,1.123651,1.123651
502,2019-12-31,IN1520190101RPF,1.041761,1.041761
753,2019-12-31,INE036D08015SD,3.329692,3.329692
1004,2019-12-31,INE095H07014PSUTF,4.649538,4.649538


In [73]:
##Renaming rates to spread_rates
spread_rates =  spread_rates.rename(columns = {'rate':'spread_rates'})
spread_rates.head()

,date,instrument,spread_rates,rate_stressed
0,2019-12-31,IN1520190101NCD,5.443980,5.443980
251,2019-12-31,IN1520190101PF,1.123651,1.123651
502,2019-12-31,IN1520190101RPF,1.041761,1.041761
753,2019-12-31,INE036D08015SD,3.329692,3.329692
1004,2019-12-31,INE095H07014PSUTF,4.649538,4.649538


In [74]:
s_rate_df = spread_rates[['instrument', 'spread_rates']].reset_index(drop=True)

In [75]:
s_rate_df

,instrument,spread_rates
0,IN1520190101NCD,5.443980
1,IN1520190101PF,1.123651
2,IN1520190101RPF,1.041761
3,INE036D08015SD,3.329692
4,INE095H07014PSUTF,4.649538
5,INE134E08925PSUT,0.602718


In [76]:
tx_bond_revamp = tx_bond_new.merge(s_rate_df, on ='instrument', how='left',sort = True)

In [77]:
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,NaN
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,NaN
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,NaN
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,NaN
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,NaN
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,NaN
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,IN1520190101NCD_3,2020-04-20,2022-08-15,2022-08-15,INRZeroCurve_200yr,INRZeroCurve_225yr,5.776,5.776,5.776000,5.443980
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,IN1520190101NCD_4,2020-04-20,2023-08-15,2023-08-15,INRZeroCurve_300yr,INRZeroCurve_325yr,6.357,6.357,6.357000,5.443980


In [78]:
#Replacing NAN's with 0's
tx_bond_revamp['spread_rates'] = tx_bond_revamp['spread_rates'].fillna(0)
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,key,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,IN0020100015CGS_1,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,IN0020100015CGS_2,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,IN0020100015CGS_3,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,IN0020100015CGS_4,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,IN0020100015CGS_5,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,IN0020100015CGS_6,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,IN1520190101NCD_1,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,IN1520190101NCD_2,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,IN1520190101NCD_3,2020-04-20,2022-08-15,2022-08-15,INRZeroCurve_200yr,INRZeroCurve_225yr,5.776,5.776,5.776000,5.443980
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,IN1520190101NCD_4,2020-04-20,2023-08-15,2023-08-15,INRZeroCurve_300yr,INRZeroCurve_325yr,6.357,6.357,6.357000,5.443980


In [79]:
tx_bond_revamp.shape

(17, 39)

In [80]:
tx_bond_revamp.columns

Index(['id', 'instrument', 'instrument_id', 'k', 'currency', 'currency_base',
       'Cashflow', 'Cashflow_Type', 'Maturity_Amount', 'Face_value', 'curve',
       'current_date', 'Cash_flow_Dates', 'holiday', 'Actual_Cash_flow_date',
       'maturity_date', 'residual_maturity_expected',
       'residual_maturity_Actual', 'lower_tenor', 'upper_tenor',
       'lower_tenor_val', 'upper_tenor_val', 'lower_maturity',
       'upper_maturity', 'split_curve', 'accounting_book',
       'accounting_portfolio', 'mark_up_value', 'intbasis', 'key', 'c_date',
       'm_date_E', 'm_date_A', 'lower_tenor_key', 'upper_tenor_key',
       'lower_rates', 'upper_rates', 'Interpolated_rates', 'spread_rates'],
      dtype='object')

In [81]:
#YTM(Yield to Maturity)
tx_bond_revamp['YTM']= tx_bond_revamp['Interpolated_rates'] + tx_bond_revamp['spread_rates'] + tx_bond_revamp['mark_up_value'] 

In [82]:
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,c_date,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,2020-04-20,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,2020-04-20,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,2020-04-20,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,2020-04-20,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,2020-04-20,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,2020-04-20,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,2020-04-20,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,2020-04-20,2022-08-15,2022-08-15,INRZeroCurve_200yr,INRZeroCurve_225yr,5.776,5.776,5.776000,5.443980,11.219980
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,2020-04-20,2023-08-15,2023-08-15,INRZeroCurve_300yr,INRZeroCurve_325yr,6.357,6.357,6.357000,5.443980,11.800980


In [83]:
#result = tx_bond_revamp.dtypes
#result

In [84]:
tx_bond_revamp['maturity_period_E'] = (tx_bond_revamp['m_date_E'] - tx_bond_revamp['c_date']).dt.days
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,m_date_E,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,2020-08-03,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,2021-02-03,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,2021-08-03,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,2022-02-03,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,2022-08-03,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,2020-08-15,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,2021-08-15,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,2022-08-15,2022-08-15,INRZeroCurve_200yr,INRZeroCurve_225yr,5.776,5.776,5.776000,5.443980,11.219980,847
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,2023-08-15,2023-08-15,INRZeroCurve_300yr,INRZeroCurve_325yr,6.357,6.357,6.357000,5.443980,11.800980,1212


In [85]:
tx_bond_revamp['maturity_period_A'] = (tx_bond_revamp['m_date_A'] - tx_bond_revamp['c_date']).dt.days
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,m_date_A,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,2020-08-03,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105,105
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,2021-02-03,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289,289
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,2021-08-03,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470,470
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,2022-02-03,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654,654
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,2022-08-03,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,2020-08-16,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117,118
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,2021-08-15,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482,482
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,2022-08-15,INRZeroCurve_200yr,INRZeroCurve_225yr,5.776,5.776,5.776000,5.443980,11.219980,847,847
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,2023-08-15,INRZeroCurve_300yr,INRZeroCurve_325yr,6.357,6.357,6.357000,5.443980,11.800980,1212,1212


In [86]:
##Condition

In [87]:
tx_bond_revamp['no_of_days_in_a_year'] = ''
for i in range (len(tx_bond_revamp)):

  
  if  tx_bond_revamp['intbasis'][i] == 0:
        tx_bond_revamp['no_of_days_in_a_year'][i] = 360
  
  elif tx_bond_revamp['intbasis'][i] == 1: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 365  
  
  elif tx_bond_revamp['intbasis'][i] == 2: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 360 
  
  elif tx_bond_revamp['intbasis'][i] == 3: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 365 
  
  elif tx_bond_revamp['intbasis'][i] == 4: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 360
   
  else:
        pass
  

/tmp/ipykernel_326/2565406497.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tx_bond_revamp['no_of_days_in_a_year'][i] = 365
/tmp/ipykernel_326/2565406497.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tx_bond_revamp['no_of_days_in_a_year'][i] = 360


In [88]:
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,lower_tenor_key,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105,105,365
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,INRZeroCurve_075m,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289,289,365
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,INRZeroCurve_125yr,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470,470,365
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,INRZeroCurve_175yr,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654,654,365
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835,365
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,INRZeroCurve_225yr,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835,365
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,INRZeroCurve_025m,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117,118,360
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,INRZeroCurve_100yr,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482,482,360
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,INRZeroCurve_200yr,INRZeroCurve_225yr,5.776,5.776,5.776000,5.443980,11.219980,847,847,360
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,INRZeroCurve_300yr,INRZeroCurve_325yr,6.357,6.357,6.357000,5.443980,11.800980,1212,1212,360


In [89]:
tx_bond_revamp['cashflow_mtm'] = tx_bond_revamp['Cashflow'] / ((1+(tx_bond_revamp['YTM']/100))**(tx_bond_revamp['residual_maturity_Actual']/tx_bond_revamp['no_of_days_in_a_year'])) 

In [90]:
tx_bond_revamp

,id,instrument,instrument_id,k,currency,currency_base,Cashflow,Cashflow_Type,Maturity_Amount,Face_value,...,upper_tenor_key,lower_rates,upper_rates,Interpolated_rates,spread_rates,YTM,maturity_period_E,maturity_period_A,no_of_days_in_a_year,cashflow_mtm
0,CGS001,IN0020100015CGS,16807,1,INR,INR,4000,C,100000,100,...,INRZeroCurve_050m,5.040,5.170,5.060000,0.000000,5.060000,105,105,365,3943.601917
1,CGS001,IN0020100015CGS,16807,2,INR,INR,4000,C,100000,100,...,INRZeroCurve_100yr,5.190,5.190,5.190000,0.000000,5.190000,289,289,365,3842.917306
2,CGS001,IN0020100015CGS,16807,3,INR,INR,4000,C,100000,100,...,INRZeroCurve_150yr,5.190,6.110,5.331538,0.000000,5.331538,470,470,365,3741.210574
3,CGS001,IN0020100015CGS,16807,4,INR,INR,4000,C,100000,100,...,INRZeroCurve_200yr,5.190,5.776,5.285543,0.000000,5.285543,654,654,365,3647.372389
4,CGS001,IN0020100015CGS,16807,5,INR,INR,4000,C,100000,100,...,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835,365,3517.792617
5,CGS001,IN0020100015CGS,16807,6,INR,INR,100000,P,100000,100,...,INRZeroCurve_250yr,5.776,5.776,5.776000,0.000000,5.776000,835,835,365,87944.81543
6,NCD001,IN1520190101NCD,16806,1,INR,INR,12750,C,150000,100,...,INRZeroCurve_050m,5.040,5.170,5.077143,5.443980,10.521123,117,118,360,12338.710772
7,NCD001,IN1520190101NCD,16806,2,INR,INR,12750,C,150000,100,...,INRZeroCurve_125yr,5.190,5.190,5.190000,5.443980,10.633980,482,482,360,11136.489436
8,NCD001,IN1520190101NCD,16806,3,INR,INR,12750,C,150000,100,...,INRZeroCurve_225yr,5.776,5.776,5.776000,5.443980,11.219980,847,847,360,9927.783236
9,NCD001,IN1520190101NCD,16806,4,INR,INR,12750,C,150000,100,...,INRZeroCurve_325yr,6.357,6.357,6.357000,5.443980,11.800980,1212,1212,360,8758.101185


In [91]:
##Cross_check the values of each columns whether are in correct order ir n

In [92]:
df2 = tx_bond_revamp.groupby('id')['cashflow_mtm'].sum()
mtm = list(df2)
mtm = [round(item, 0) for item in mtm]
mtm

[106638.0, 145198.0, 210214.0]

In [93]:
def get_mtm_bc(tx_eq):
    
    res_df = list()
    for k, h in tx_eq.groupby('id'):
        result = {'idx':h["id"].iloc[0] , "env":k,"instrument":h["instrument"].iloc[0] ,"currency":h["currency"].iloc[0] ,"currency_base":h["currency_base"].iloc[0], "face_value":h["Face_value"].iloc[0], "maturity_amount":h["Maturity_Amount"].iloc[0]}
        res_df.append(result)
        #res_df = pd.DataFrame(res_df)

    return res_df

In [94]:
get_mtm_bc(tx_bond_revamp)

[{'idx': 'CGS001',
  'env': 'CGS001',
  'instrument': 'IN0020100015CGS',
  'currency': 'INR',
  'currency_base': 'INR',
  'face_value': 100,
  'maturity_amount': 100000},
 {'idx': 'NCD001',
  'env': 'NCD001',
  'instrument': 'IN1520190101NCD',
  'currency': 'INR',
  'currency_base': 'INR',
  'face_value': 100,
  'maturity_amount': 150000},
 {'idx': 'PF001',
  'env': 'PF001',
  'instrument': 'IN1520190101PF',
  'currency': 'INR',
  'currency_base': 'INR',
  'face_value': 100,
  'maturity_amount': 200000}]

In [95]:
#, "bondwise_current_mtm":mtm[0]
red = get_mtm_bc(tx_bond_revamp)
res = pd.DataFrame(red)
res

,idx,env,instrument,currency,currency_base,face_value,maturity_amount
0,CGS001,CGS001,IN0020100015CGS,INR,INR,100,100000
1,NCD001,NCD001,IN1520190101NCD,INR,INR,100,150000
2,PF001,PF001,IN1520190101PF,INR,INR,100,200000


In [97]:
mtm = pd.DataFrame(mtm)

In [98]:
mtm

,0
0,106638.0
1,145198.0
2,210214.0


In [99]:
mtm.rename(columns = {0:'bondwise_current_mtm'}, inplace = True)
mtm = mtm.reset_index(drop = True)

In [100]:
result_new = pd.concat([res, mtm], axis = 1)

In [101]:
result_new

,idx,env,instrument,currency,currency_base,face_value,maturity_amount,bondwise_current_mtm
0,CGS001,CGS001,IN0020100015CGS,INR,INR,100,100000,106638.0
1,NCD001,NCD001,IN1520190101NCD,INR,INR,100,150000,145198.0
2,PF001,PF001,IN1520190101PF,INR,INR,100,200000,210214.0


In [102]:
#result_new$price <- (result_new$bondwise_current_mtm * result_new$face_value) / result_new$maturity_amount
result_new['price'] = (result_new['bondwise_current_mtm']*result_new['face_value'])/result_new['maturity_amount']

In [103]:
result_new

,idx,env,instrument,currency,currency_base,face_value,maturity_amount,bondwise_current_mtm,price
0,CGS001,CGS001,IN0020100015CGS,INR,INR,100,100000,106638.0,106.638000
1,NCD001,NCD001,IN1520190101NCD,INR,INR,100,150000,145198.0,96.798667
2,PF001,PF001,IN1520190101PF,INR,INR,100,200000,210214.0,105.107000


### All MTMS

In [104]:
interpolated_curve_rate = np.empty((n+1,1))
lW_rate1 = np.empty((n+1,3))
up_rate1 = np.empty((n+1,3))

In [105]:
simulated_interest

,date,curve,tenor,rate,rate_stressed,key,simple_returns,simulated rates
0,2019-12-31,INRZeroCurve,025m,5.040,5.040,INRZeroCurve_025m,0.040,5.040
1,2019-12-30,INRZeroCurve,025m,5.000,5.000,INRZeroCurve_025m,0.040,5.080
2,2019-12-27,INRZeroCurve,025m,4.960,4.960,INRZeroCurve_025m,-0.050,5.080
3,2019-12-26,INRZeroCurve,025m,5.010,5.010,INRZeroCurve_025m,-0.010,4.990
4,2019-12-24,INRZeroCurve,025m,5.020,5.020,INRZeroCurve_025m,0.030,5.030
...,...,...,...,...,...,...,...,...
246,2018-12-26,INRZeroCurve,9yr,7.371,7.371,INRZeroCurve_9yr,-0.002,6.760
247,2018-12-24,INRZeroCurve,9yr,7.373,7.373,INRZeroCurve_9yr,0.035,6.782
248,2018-12-21,INRZeroCurve,9yr,7.338,7.338,INRZeroCurve_9yr,0.023,6.819
249,2018-12-20,INRZeroCurve,9yr,7.315,7.315,INRZeroCurve_9yr,0.000,6.807


In [156]:
for i in range(0,len(tx_bond_revamp)):
    a = tx_bond_revamp.loc[i,['lower_tenor_key', 'upper_tenor_key']]
    a = list(a)
    rates = simulated_interest[simulated_interest['key'].isin(a)]
    rates1 = rates.loc[:, ['key','simulated rates']]
    rates1 = rates1.set_index('key')
    ##Segregating simulations according to the instrument
    rates2 = (rates1.assign(labels = rates1.groupby(level = 0).cumcount())
            .groupby([rates1.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))
    rates2 = rates2.transpose()
    
    #rates = pd.merge(simulated_interest,a,how='left',left_on='key',right_on='lower_tenor_key')
   # pd.merge(tenor_df,simulated_interest,how='left',left_on='tenor_list',right_on='key').drop(['key','date','rate_stressed','simple_returns','rate'], axis=1)
    

In [168]:
import re
for i in range (0, len(rates2)):
    re.search(rates2['INRZeroCurve_050m'][i],rates2['INRZeroCurve_075m'][i])

TypeError: first argument must be string or compiled pattern

In [167]:
rates2['INRZeroCurve_050m'][0]

5.17

In [163]:
type(rates2)

pandas.core.frame.DataFrame

In [157]:
rates2

key,INRZeroCurve_050m,INRZeroCurve_075m
simulated rates,5.17,5.19
simulated rates,5.14,5.16
simulated rates,5.20,5.15
simulated rates,5.18,5.20
simulated rates,5.13,5.15
...,...,...
simulated rates,5.19,5.19
simulated rates,5.19,5.19
simulated rates,5.15,5.19
simulated rates,5.15,5.19


In [129]:
rates

,date,curve,tenor,rate,rate_stressed,key,simple_returns,simulated rates
0,2019-12-31,INRZeroCurve,050m,5.17,5.17,INRZeroCurve_050m,-0.03,5.17
1,2019-12-30,INRZeroCurve,050m,5.20,5.20,INRZeroCurve_050m,0.03,5.14
2,2019-12-27,INRZeroCurve,050m,5.17,5.17,INRZeroCurve_050m,0.01,5.20
3,2019-12-26,INRZeroCurve,050m,5.16,5.16,INRZeroCurve_050m,-0.04,5.18
4,2019-12-24,INRZeroCurve,050m,5.20,5.20,INRZeroCurve_050m,0.01,5.13
...,...,...,...,...,...,...,...,...
246,2018-12-26,INRZeroCurve,075m,6.12,6.12,INRZeroCurve_075m,0.00,5.19
247,2018-12-24,INRZeroCurve,075m,6.12,6.12,INRZeroCurve_075m,0.00,5.19
248,2018-12-21,INRZeroCurve,075m,6.12,6.12,INRZeroCurve_075m,0.00,5.19
249,2018-12-20,INRZeroCurve,075m,6.12,6.12,INRZeroCurve_075m,0.00,5.19


In [ ]:
##Sorting the dataframe lower_rate wise
#tx_bond_revamp.sort_values(by=['lower_rates'])

In [ ]:
result_new$price <- (result_new$bondwise_current_mtm * result_new$face_value) / result_new$maturity_amount


In [ ]:
if (decay_factor==0){
  
  Bondwise_VaR<- ddply(total_change_in_MTM, "instrument", function(x){
    change_in_MTM_sorted=sort(x$value[2:length(x$value)],decreasing = T)
    
    VaR_exact_one_day = -quantile(change_in_MTM_sorted, 1-confidence_level)
    
    nearby   <- floor(confidence_level * length(change_in_MTM_sorted))
    nearby_nxt <- nearby + 1
    
    diff1 <- as.numeric(VaR_exact_one_day + change_in_MTM_sorted[nearby])
    diff2 <- as.numeric(-change_in_MTM_sorted[nearby_nxt] - VaR_exact_one_day)
    
    if(diff1 == diff2)
    {
      VaR_nearby_one_day= - change_in_MTM_sorted[nearby + 1]
    }
    
    if(diff1 < diff2)
    {
      VaR_nearby_one_day = -change_in_MTM_sorted[nearby]
    }else
    {
      VaR_nearby_one_day = -change_in_MTM_sorted[nearby + 1]
    }
    
    return(data.frame(id=seq(1,length(unique(x$instrument))), instrument=unique(x$instrument), Decay_factor=decay_factor, Holding_period=holding_period, exact_one_day_VaR=VaR_exact_one_day, nearby_one_day_VaR=VaR_nearby_one_day))
  })
} else {


In [ ]:
if decay_factor == 0:
    
    change_in_MTM_sorted = sorted(total_change_in_mtm, reverse = True)
    VaR_exact_one_day = -(np.quantile(total_change_in_MTM, 0.01))
    nearby = np.floor(confidence_level * len(change_in_MTM_sorted))
    nearby = int(nearby)
    nearby_nxt = nearby + 1
    
    diff1 = pd.to_numeric(VaR_exact_one_day + change_in_MTM_sorted[nearby])
    diff2 = pd.to_numeric(- change_in_MTM_sorted[nearby_nxt] - VaR_exact_one_day)
    
    if (diff1 == diff2):
        VaR_nearby_one_day = - change_in_MTM_sorted[nearby + 1]
    elif (diff1 < diff2):
        VaR_nearby_one_day = - change_in_MTM_sorted[nearby]
    else:
        VaR_nearby_one_day = - change_in_MTM_sorted[nearby + 1]
        
else:
    #decay_weights <- ((1-decay_factor)* c(decay_factor^(0:(length(unique(x$env))-2)))/(1-decay_factor^((length(unique(x$env))-1))))
    
    decay_weights = (1-decay_factor)* ((decay_factor ** range(0,(len(change_df['idx'].unique())-1)))/((1 - decay_factor) ** (len(change_df['idx'].unique() -1))))
    total_mtm = change_df.groupby('idx')[['mtm_bc']].sum
    mtm_returns = []

    for i in range(1,(len(change_df['idx'].unique() -1))):
                   fv = total_mtm['mtm_bc'][0]
                   returns = (total_mtm['mtm_bc'][i] - fv)/fv
                   mtm_returns.append(returns) 
                    
    decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
    mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])
    mtm = pd.concat([mtm_returns,decay_weights], axis = 1)
    mtm = mtm.sort_values(['mtm_returns'])
    mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])
    a = 1-confidence_level
    mtm1 = mtm.loc[mtm['cum_decay_weights'] < a]
                                         
    if (len(mtm1)==0):
                  VaR_exact_equity_hp = np.abs(- mtm['mtm_returns'][0] * total_mtm['mtm_bc'][0])

                  VaR_nearby_equity_hp = VaR_exact_equity_hp

    else:
                  index = len(mtm1['cum_decay_weights'])
                                         
                  interpolated_return_exact = mtm['mtm_returns'][index] + ((mtm['mtm_returns'][index + 1] - mtm['mtm_returns'][index]) * ((1 - confidence_level) - mtm['cum_decay_weights'][index]) / (mtm['cum_decay_weights'][index + 1] - mtm['cum_decay_weights'][index] ))
    
                  VaR_exact_equity_hp =  ((total_mtm[1] * (interpolated_return_exact)))
    
                  if((((1 - confidence_level) - mtm['cum_decay_weights'][index]) < (mtm['cum_decay_weights'][index + 1] - (1 - confidence_level) ))):
            
                            interpolated_return_nearby = mtm['mtm_returns'][index]
                  else:
                            index = len(mtm1['cum_decay_weights'])
                            interpolated_return_nearby = mtm['mtm_returns'][index + 1]
    
                            VaR_nearby_equity_hp = np.abs(total_mtm['mtm_bc'][0] * (interpolated_return_nearby))
                                         
                                         
    

In [ ]:
Bondwise_VaR<- ddply(Result, "instrument", function(x){
    
    decay_weights <- ((1-decay_factor)* c(decay_factor^(0:(length(unique(x$env))-2)))/(1-decay_factor^((length(unique(x$env))-1))))
    
    total_mtm <- tapply(x$all_mtm,x$env, sum)
    
    mtm_returns <- NULL
    
    for(i in 1 : (length(unique(Result$env))-1))
    {
      
      mtm_returns[i] = (total_mtm[i + 1] - total_mtm[1])/total_mtm[1]
      
    }
    
    mtm <- data.frame(mtm_returns,decay_weights)
    mtm1 <- mtm[order(mtm_returns),]
    mtm1$cum_decay_weights <- cumsum(mtm1$decay_weights)
    
    mtm <- data.frame(mtm_returns,decay_weights)
    mtm <- mtm[order(mtm_returns),]
    mtm$cum_decay_weights <- cumsum(mtm$decay_weights)
    
    mtm1 <- subset(mtm, cum_decay_weights < 1 - confidence_level)
    
    if(length(mtm1$cum_decay_weights) == 0)
    {
      
      VaR_exact_one_day <- abs(- mtm$mtm_returns[1] * total_mtm[1])
      
      VaR_nearby_one_day <- VaR_exact_one_day
      
      
    }else
      
    {
      
      index <- length(mtm1$cum_decay_weights)
      
      # _________________________________________________________________________________________________________
      
      # Exact Percentile
      
      interpolated_return_exact <- mtm$mtm_returns[index] + ((mtm$mtm_returns[index + 1] - mtm$mtm_returns[index]) * ((1 - confidence_level) - mtm$cum_decay_weights[index]) / (mtm$cum_decay_weights[index + 1] - mtm$cum_decay_weights[index]))
      
      VaR_exact_one_day <-  abs(total_mtm[1] * (interpolated_return_exact))
      
      # _________________________________________________________________________________________________________
      
      # Nearby Percentile
      
      if (((1-confidence_level)- mtm$cum_decay_weights[index]) < (mtm$cum_decay_weights[index + 1] - (1-confidence_level)))
        
      {
        interpolated_return_nearby <- mtm$mtm_returns[index]
      }else
        
      {
        interpolated_return_nearby <- mtm$mtm_returns[index + 1]
      }
      
      VaR_nearby_one_day <- abs(total_mtm[1] * (interpolated_return_nearby))
      
    }
    return(data.frame(id=seq(1,length(unique(x$instrument))), instrument=unique(x$instrument), Decay_factor=decay_factor, Holding_period=holding_period, exact_one_day_VaR=VaR_exact_one_day, nearby_one_day_VaR=VaR_nearby_one_day))
  }) 
}
